# CDD Streaming

In [18]:
!pip3 install opencv-python  > /dev/null
!pip3 install albumentations > /dev/null

^C
ERROR: Operation cancelled by user


In [ ]:
import cv2 as cv
import albumentations as a


def get_training_augmentation(_src, _bboxes, _category_id, _size):
    transform = a.Compose([a.Resize(_size[0], _size[1])], bbox_params=a.BboxParams(format='coco', label_fields=['category_id']))
    return transform(image=_src, bboxes=_bboxes, category_id=_category_id)['bboxes']


# BGR
BOX_COLOR = (0, 0, 225) # Red
BOX_THICKESS = 5
TEXT_COLOR = (255, 255, 255) # White
TEXT_SIZE = 0.9
TEXT_FONT = 0
IMG_SIZE = (200, 200)

category_id_to_name = { 1: "dent", 2: "broken_glass", 3: "deflated_wheel", 4: "scratch", 5: "broken_headlight"}

def show_webcam(mirror=False):
    cam = cv.VideoCapture(0)
    for fps in range(5000):
        _, img = cam.read()
        #img = cv.resize(img, (round(200+fps), round(200+fps)))
        size = img.shape

##########
###Back###
##########
        img_crop = cv.resize(img, IMG_SIZE)
        #########################################################
        # bbox, category_id = model.detect(img_crop) 
        bbox = [10, 10, 150, 150]
        category_id = [1]
        #########################################################
        transformed_bboxes = get_training_augmentation( img_crop,       \
                                                        [bbox],         \
                                                        [category_id],  \
                                                        size            \
                                                        )
##########
###Back###
##########

        if mirror: 
            img = cv.flip(img, 1)

        for left, top, right, bottom in transformed_bboxes:
            left, top, right, bottom = int(left), int(top), int(right), int(bottom)
            cv.rectangle(img,(left,top),(right, bottom), BOX_COLOR, BOX_THICKESS)
            cv.putText(img, category_id_to_name.get(category_id[0]), (right+10, bottom), TEXT_FONT, TEXT_SIZE, TEXT_COLOR)

        cv.imshow('my webcam', img)
        if cv.waitKey(1) == 27: 
            break  # esc to quit
    cv.destroyAllWindows()

show_webcam(mirror=True)

KeyboardInterrupt: 

# Face Detection Streaming

In [ ]:
!pip3 install dlib             > /dev/null
!pip3 install face_recognition > /dev/null
!pip3 install opencv-python    > /dev/null

In [ ]:
import face_recognition

<module 'face_recognition' from '/Users/alexgiving/opt/anaconda3/lib/python3.8/site-packages/face_recognition/__init__.py'>


In [ ]:
# import libraries
import cv2 as cv
import face_recognition

BOX_COLOR = (0, 0, 225) # Red
BOX_THICKESS = 2
TEXT_COLOR = (255, 255, 255) # White
TEXT_SIZE = 0.9
TEXT_FONT = 0
IMG_SIZE = (200, 200)


video_capture = cv.VideoCapture(0)
face_locations = []
 
for fps in range(500):
    ret, frame = video_capture.read()
    frame = cv.flip(frame, 1)      # mirror
    rgb_frame = frame[:, :, ::-1]  # to RGB

    face_locations = face_recognition.face_locations(rgb_frame)
    
    for top, right, bottom, left in face_locations:
        cv.rectangle(frame, (left, top), (right, bottom), BOX_COLOR, BOX_THICKESS)
        cv.putText(frame, "dolboeb", (right+10, bottom), TEXT_FONT, TEXT_SIZE, TEXT_COLOR)


    cv.imshow('Video', frame)      
    if cv.waitKey(1) == 27: 
        break  # esc to quit
 
video_capture.release()
cv.destroyAllWindows()

KeyboardInterrupt: 

# Car Detection Streaming

In [3]:
BOX_COLOR = (0, 0, 225) # Red
BOX_THICKESS = 2
TEXT_COLOR = (255, 255, 255) # White
TEXT_SIZE = 0.9
TEXT_FONT = 0
IMG_SIZE = (200, 200)

In [4]:
import numpy as np
from PIL import Image

def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return Image.fromarray(tensor)

In [5]:
from torchvision import transforms as T

def get_transform(src, train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    transform = T.Compose(transforms)
    return transform(image=src)

In [6]:
import cv2 as cv
import torch
import torch.utils.data

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


PATH = "model_70.pt"
model = torch.load(PATH, map_location='cpu')
model.eval()

cam = cv.VideoCapture("car.jpeg")
ret, frame = cam.read()
assert(ret)


In [7]:
img_ = torch.tensor(frame)
img = tensor_to_image(img_)

transform = T.Compose([T.ToTensor()])
img_T = transform(img)

In [1]:
with torch.no_grad():
    prediction = model([img_T])

NameError: name 'torch' is not defined

In [ ]:
prediction[0]['boxes']

NameError: name 'prediction' is not defined

In [ ]:
for top, right, bottom, left in prediction[0]['boxes']:
    print(top, right, bottom, left)

NameError: name 'prediction' is not defined

In [ ]:
for top, right, bottom, left in prediction[0]['boxes']:
    top = int(top)
    right = int(right)
    bottom = int(bottom)
    left = int(left)
    cv.rectangle(frame, (left, top), (right, bottom), BOX_COLOR, BOX_THICKESS)
    cv.putText(frame, "dolboeb", (right+10, bottom), TEXT_FONT, TEXT_SIZE, TEXT_COLOR)


In [88]:
cv.imshow('graycsale image',frame)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
#cv.imshow('graycsale image',frame)
#cv.waitKey(0)
#cv.destroyAllWindows()

# Master

import cv2 as cv
import albumentations as a


def get_training_augmentation(_src, _bboxes, _category_id, _size):
    transform = a.Compose([a.Resize(_size[0], _size[1])], bbox_params=a.BboxParams(format='coco', label_fields=['category_id']))
    return transform(image=_src, bboxes=_bboxes, category_id=_category_id)['bboxes']


# BGR
BOX_COLOR = (0, 0, 225) # Red
BOX_THICKESS = 5
TEXT_COLOR = (255, 255, 255) # White
TEXT_SIZE = 0.9
TEXT_FONT = 0
IMG_SIZE = (200, 200)

category_id_to_name = { 1: "dent", 2: "broken_glass", 3: "deflated_wheel", 4: "scratch", 5: "broken_headlight"}

def show_webcam(mirror=False):
    cam = cv.VideoCapture(0)
    for fps in range(5000):
        _, img = cam.read()
        #img = cv.resize(img, (round(200+fps), round(200+fps)))
        size = img.shape

##########
###Back###
##########
        img_crop = cv.resize(img, IMG_SIZE)
        #########################################################
        # bbox, category_id = model.detect(img_crop) 
        bbox = [10, 10, 150, 150]
        category_id = [1]
        #########################################################
        transformed_bboxes = get_training_augmentation( img_crop,       \
                                                        [bbox],         \
                                                        [category_id],  \
                                                        size            \
                                                        )
##########
###Back###
##########

        if mirror: 
            img = cv.flip(img, 1)

        for left, top, right, bottom in transformed_bboxes:
            left, top, right, bottom = int(left), int(top), int(right), int(bottom)
            cv.rectangle(img,(left,top),(right, bottom), BOX_COLOR, BOX_THICKESS)
            cv.putText(img, category_id_to_name.get(category_id[0]), (right+10, bottom), TEXT_FONT, TEXT_SIZE, TEXT_COLOR)

        cv.imshow('my webcam', img)
        if cv.waitKey(1) == 27: 
            break  # esc to quit
    cv.destroyAllWindows()

show_webcam(mirror=True)